# 🛡️ HazardSAFE: Agentic AI for HazMat Safety

**Google Agentic AI Hackathon Submission**

HazardSAFE is an autonomous multi-agent system designed to automate compliance, tracking, and certification for Hazardous Materials (HazMat) transport. It leverages the **Google Agentic Stack** (Gemini, LangChain, LangGraph) to create a "Glass Box" compliance engine that is transparent, verifiable, and safe.

## 🚀 Key Features
1.  **Compliance Agent**: Uses RAG + Code-as-Policy to validate scenarios against complex regulations.
2.  **HITL Workflow**: A robust Human-in-the-Loop approval system for high-risk decisions.
3.  **Provenance Agent**: Logs every action to an immutable ledger (Firestore) for auditability.
4.  **Verifiable Credentials**: Issues W3C-standard credentials for compliant shipments.

---

## 1. Setup & Configuration
First, we install dependencies and configure the environment.

In [ ]:
!pip install -q google-generativeai chromadb google-cloud-firestore langflow streamlit graphviz pandas

import os
import sys
import json
import time

# Add project root to path
sys.path.append('../')

# Configure API Key (Use Kaggle Secrets or env var)
if "GOOGLE_API_KEY" not in os.environ:
    # from kaggle_secrets import UserSecretsClient
    # user_secrets = UserSecretsClient()
    # os.environ["GOOGLE_API_KEY"] = user_secrets.get_secret("GOOGLE_API_KEY")
    print("⚠️ GOOGLE_API_KEY not found. Agents will run in MOCK MODE.")
else:
    print("✅ API Key configured.")

## 2. Initialize Agents
We initialize the core agents: Compliance, Provenance, and the Workflow Manager.

In [ ]:
from src.agents.compliance_agent import HazardComplianceAgent
from src.agents.provenance_agent import HazardProvenanceAgent
from src.workflow.manager import WorkflowManager

# Initialize
compliance_agent = HazardComplianceAgent()
provenance_agent = HazardProvenanceAgent()
workflow_manager = WorkflowManager()

print("🤖 Agents Initialized successfully.")

## 3. Define a HazMat Scenario
Let's simulate a shipment of **Radioactive Material (Class 7)**. This requires strict temperature controls.

In [ ]:
scenario = {
    "id": "SCN-KAGGLE-001",
    "material_class": "Class 7",
    "package_type": "Type B(U)",
    "ambient_temperature_c": 35.0,  # Limit is usually 38C
    "transport_index": 0.8
}

print(f"📄 Scenario Defined:\n{json.dumps(scenario, indent=2)}")

## 4. AI Compliance Check (RAG + Code-as-Policy)
The Compliance Agent retrieves regulations and generates Python code to validate the scenario.

In [ ]:
print("🕵️ Compliance Agent analyzing...")
decision = compliance_agent.check_scenario(scenario)

print(f"\n💡 Decision: {'✅ Compliant' if decision['compliant'] else '❌ Non-Compliant'}")
print(f"   Reason: {decision['reason']}")

## 5. Human-in-the-Loop (HITL) Workflow
For high-risk materials, we don't trust AI blindly. The workflow enters a `PENDING_HITL` state, requiring human approval.

In [ ]:
# Create Workflow
wf_id = workflow_manager.create_workflow(scenario['id'], scenario)

# Trigger HITL
print(f"🛑 Triggering HITL for Workflow {wf_id}...")
workflow_manager.trigger_hitl(wf_id, decision)

# Check Status
state = workflow_manager.get_workflow_state(wf_id)
print(f"   Current Status: {state['status']}")

# Simulate Human Approval
print("\n👤 Human Reviewer Approving...")
workflow_manager.approve_workflow(wf_id, "kaggle_judge@hazardsafe.ai", "Looks good, temperature within limits.")

# Verify Final State
final_state = workflow_manager.get_workflow_state(wf_id)
print(f"✅ Final Status: {final_state['status']}")
print(f"   Reviewer: {final_state['human_reviewer']}")

## 6. Provenance & Audit Trail
Every step is logged to an immutable ledger. This ensures accountability.

In [ ]:
# Log the final approval event
provenance_agent.log_event(
    "HITL_APPROVED", 
    "KaggleNotebook", 
    {"workflow_id": wf_id, "user": "kaggle_judge"}
)

# Retrieve History (Mock)
print("\n📜 Audit Trail (Recent Events):")
history = provenance_agent.get_provenance_graph()
for event in history[-3:]:
    print(f" - [{event['type']}] Agent: {event['agent_id']}")

## 7. Evaluation Metrics
We evaluate the system's accuracy and robustness using a batch of test scenarios.

In [ ]:
import random

def run_evaluation(num_samples=10):
    correct = 0
    print(f"🧪 Running evaluation on {num_samples} scenarios...")
    
    for i in range(num_samples):
        # Generate random temp
        temp = random.uniform(20, 50)
        expected = temp <= 38.0
        
        test_scenario = scenario.copy()
        test_scenario['ambient_temperature_c'] = temp
        
        decision = compliance_agent.check_scenario(test_scenario)
        
        if decision['compliant'] == expected:
            correct += 1
            
    accuracy = (correct / num_samples) * 100
    print(f"\n✅ Evaluation Complete")
    print(f"   Accuracy: {accuracy:.1f}%")
    return accuracy

run_evaluation()